In [4]:
import sys
import importlib
sys.path.insert(0, '../src/')

In [5]:
import chatbot.dataset.messenger
importlib.reload(chatbot.dataset.messenger)

m = chatbot.dataset.messenger.load_conversations('/home/patz/Heap/messages/')

In [6]:
for c in m:
    c.rename('Patztablook TwentyTwo', 'patz')

In [11]:
m = list(filter(lambda c: len(c) >= 64, m))
print(len(m))
for c in m:
    print(c.participants, len(c))

53
{'Sigismund Häggkvist', 'Robin Stringer', 'patz', 'Martin McNickle', 'Rawen Dorfin'} 15750
{'patz', 'Amélie Novotná'} 156
{'patz', 'Ondřej Smolík'} 176
{'patz', 'Alexander Terziev'} 4433
{'Manka Roblíková', 'patz'} 745
{'patz', 'Jan Zasadil'} 38947
{'patz', 'Katie Alzbeta Brown'} 102
{'Jan Vaniš', 'patz'} 407
{'patz', 'Karel Loula', 'Katie Alzbeta Brown'} 183
{'patz', 'Filip Kastl'} 6845
{'Scanthning Brot', 'patz'} 407
{'Malcolm Diene', 'Štěpán Chrast', 'Jakub Tichanek', 'Patrik Kabrna', 'Šimon Dutý', 'patz', 'Petr Hošek', 'Abudi  Shalabi', 'Adéla Gabesamová', 'Adam Vanĕček', 'Adam Dvořák', 'Alexander Terziev'} 283
{'patz', 'Martina Zavoralová'} 88
{'Scanthning Brot', 'patz'} 91926
{'David Kaňovský', 'Anthony Hani', 'Jonas Kutil', 'Pomat 794', 'Štěpán Chrast', 'Jiří Podaný', 'Jakub Tichanek', 'Matěj Splavec', 'Amélie Novotná', 'patz', 'Vojta Větrovec', 'Kryštof Tahal', 'Sběratel Banánů', 'Scanthning Brot'} 3775
{'Michal Zavoral', 'patz'} 500
{'Sběratel Banánů', 'patz'} 72094
{'patz'

In [29]:
val_indices = [-1, -3, -7, -8, -9, -21, -34]
train_indices = np.delete(np.arange(len(m)), val_indices)

print(len(m))
for i in val_indices:
    print(m[i].participants)

53
{'patz', 'Adéla Karolína Žáčková'}
{'Yin Pui Lai', 'Audra Brown', 'Lamine Diene', 'Malcolm Diene', 'Jakub Tichanek', 'Jan Kotrč', 'ɹəsՈ ʞooqəɔɐℲ', 'Vincent Fournier', 'Jonathan Ogilvie', 'Kaizo Sama', 'Sběratel Banánů'}
{'Sven Bernas', 'patz'}
{'Adam Kobes', 'patz'}
{'patz', 'Jirka Vrba'}
{'patz', 'Štěpán Hanzálek'}
{'Malcolm Diene', 'patz'}


In [31]:
import chatbot.dataset.messenger
importlib.reload(chatbot.dataset.messenger)

special_tokens = {
    'conversation_start': '[CSTART]',
    'conversation_end': '[CEND]',
    'message_start': '[MSTART]',
    'message_end': '[MEND]',
    'writes': '[WRITES]',
}

chatbot.dataset.messenger.generate_corpus('../data/train.txt', [m[i] for i in train_indices], special_tokens)
chatbot.dataset.messenger.generate_corpus('../data/val.txt', [m[i] for i in val_indices], special_tokens)